In [ ]:
def calc_f1(gts, preds):
    from sklearn.metrics import f1_score

    f1_macro = f1_score(gts, preds, average='macro')
    acc = np.sum(np.array(gts) == np.array(preds)) / len(gts)
    return "acc", acc, "f1 macro", f1_macro
def extract_letter(text):
    import re
    # Regular expression to capture a single letter (A, B, C, D) inside \boxed{}
    match = re.search(r'\\boxed\{.*?\b([A-D])\b.*?\}', text)
    
    return match.group(1) if match else None

## Perspective

In [ ]:
from openai import OpenAI
import base64
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

import io
import base64
client = OpenAI()
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

def pil_to_base64(pil_image):
    img_byte_arr = io.BytesIO()
    pil_image.save(img_byte_arr, format='PNG')
    img_encoded_str = base64.b64encode(img_byte_arr.getvalue()).decode('ascii')
    return img_encoded_str


def extract_answer_from_model_response(model_response):
    model_answer = model_response.strip()
    model_answer = model_answer.replace("option", "")
    try:
        model_answer = model_answer.split('oxed{')[-1][0]
    except:
        pass
        
    return model_answer


answer_dict = {}
from datasets import load_dataset
dataset = load_dataset(f"MahtabBg/NEWPerspective")
dataset = dataset['train']
if True:
    for i in tqdm(range(len(dataset))):
        example = dataset[i]
        qid = example['id']

        question_image = example['topdown']
        choice_image = example['choices']
        question_image = pil_to_base64(question_image)
        choice_image = pil_to_base64(choice_image)
        
        
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "text",
                            "text": "<Image 1>:",
                        },

                        {
                            "type": "image_url",
                            "image_url": {
                                "url":f"data:image/jpeg;base64,{question_image}", "detail": "auto",
                            },
                        },
                        {
                            "type": "text",
                            "text": "<Image 2>:",
                        },
                        {
                            "type": "image_url",
                            "image_url": {
                                "url": f"data:image/jpeg;base64,{choice_image}", "detail": "auto",
                            },
                        },
                        {
                            "type": "text",
                            "text": "<Image 1> shows an image from the top of a scene with a red square indicating an agent and a red arrow indicating the agent's direction of view.\nSelect from the <Image 2> which image represents the agent's view. Please first solve the problem step by step, then put your final answer or a single letter (if it is a multiple choice question) in one \"\\boxed{}\"",
                        },
                        
                    ],
                }
            ],
            max_tokens=1024,
            # temperature=0,
        )
        
        answer_dict[qid] = {"response":response.choices[0].message.content , "pred": extract_answer_from_model_response(response.choices[0].message.content)}
        if len(answer_dict)%10==0:
            np.save("perspective_gpt4o.npy", answer_dict)
np.save("perspective_gpt4o.npy", answer_dict)     

In [ ]:
answer_dict= np.load("perspective_gpt4o.npy" ,allow_pickle=True).item()
from datasets import load_dataset
dataset = load_dataset(f"MahtabBg/NEWPerspective")
dataset = dataset['train']
map_dict = {"up": "A", "right": "B", "down": "C", "left": "D"}
gts, preds =[], []
for i in range(len(dataset)):
    el = dataset[i]
    gts.append(map_dict[el['answer']].lower())
    pred = extract_letter(answer_dict[el['id']]['response']).lower()
    assert pred in ['a', 'b', 'c', 'd']
    # if pred!= answer_dict[el['id']]['pred'].lower():
    #     print(pred, answer_dict[el['id']]['pred'].lower(), answer_dict[el['id']]['response'])
    preds.append(pred)

calc_f1(gts, preds)    

## Video

In [ ]:
from openai import OpenAI
import base64
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import io
import base64
client = OpenAI()
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')
        
def pil_to_base64(pil_image):
    img_byte_arr = io.BytesIO()
    pil_image.save(img_byte_arr, format='PNG')
    img_encoded_str = base64.b64encode(img_byte_arr.getvalue()).decode('ascii')
    return img_encoded_str
    
def extract_answer_from_model_response(model_response):
    model_answer = model_response.strip()
    model_answer = model_answer.replace("option", "")
    try:
        model_answer = model_answer.split('oxed{')[-1][0]
    except:
        pass
        
    return model_answer


dataset = load_dataset(f"MahtabBg/Video")
dataset = dataset['train']


answer_dict = {}
from datasets import load_dataset

if True:
    for i in tqdm(range(len(dataset))):
        example = dataset[i]
        qid = example['id']

        
       
        question_image = example['question']
        choice_image = example['choices']
        
        question_image = pil_to_base64(question_image)
        choice_image = pil_to_base64(choice_image)
        
        
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {
                    "role": "user",
                    "content": [
                         {
                            "type": "text",
                            "text": "<Image 1>:",
                        },
                        {
                            "type": "image_url",
                            "image_url": {
                                "url":f"data:image/jpeg;base64,{question_image}", "detail": "auto",
                            },
                        },
                         {
                            "type": "text",
                            "text": "<Image 2>:",
                        },
                        {
                            "type": "image_url",
                            "image_url": {
                                "url": f"data:image/jpeg;base64,{choice_image}", "detail": "auto",
                            },
                        },
                        {
                            "type": "text",
                            "text": "You see 4 sequential frames of a video in <Image 1>, but one is missing (marked with '?').\n Choose which of the images in <Image 2> correctly fills the missing frame.\nRemember, the camera only moves in one direction (left or right) in the video. Please first solve the problem step by step, then put your final answer or a single letter (if it is a multiple choice question) in one \"\\boxed{}\".",
                        },
                        
                    ],
                }
            ],
            max_tokens=1024,
            # temperature=0,
        )
       
        
        answer_dict[qid] = {"response":response.choices[0].message.content , "pred": extract_answer_from_model_response(response.choices[0].message.content)}
        if len(answer_dict)%10==0:
            np.save("video_gpt4o.npy", answer_dict)
np.save("video_gpt4o.npy", answer_dict)     
      

In [ ]:
answer_dict= np.load("video_gpt4o.npy" ,allow_pickle=True).item()
from datasets import load_dataset
dataset = load_dataset(f"MahtabBg/Video")
dataset = dataset['train']
gts, preds =[], []
for i in range(len(dataset)):
    el = dataset[i]
    gts.append(el['answer'].lower())
    pred = extract_letter(answer_dict[el['id']]['response']).lower()
    assert pred in ['a', 'b', 'c']
    # if pred!= answer_dict[el['id']]['pred'].lower():
    #     print(pred, answer_dict[el['id']]['pred'].lower(), answer_dict[el['id']]['response'])
    preds.append(pred)

calc_f1(gts, preds) 